In [1]:
%run preprocessing.ipynb

In [2]:
%run temporal_preprocess.ipynb

100%|██████████| 6/6 [04:42<00:00, 47.12s/it]


                       Feature  Importance
0                       ged_sb    0.113809
1               decay_ged_ns_1    0.098164
2      sptime_dist_k001_ged_ns    0.054703
3       splag_1_decay_ged_sb_1    0.035678
4      sptime_dist_k001_ged_os    0.032818
5       sptime_dist_k10_ged_ns    0.029221
6       sptime_dist_k10_ged_os    0.025554
7        sptime_dist_k1_ged_ns    0.024370
8           wdi_nv_agr_totl_kd    0.023648
9              spei_48_detrend    0.021733
10  ged_sb_decay_12_time_since    0.020582
11           spei1_gsm_detrend    0.019496
12                   ged_gte_1    0.018472
13       mov_avg_6_ged_best_sb    0.016851
14       mov_sum_6_ged_best_sb    0.016176
15       sptime_dist_k1_ged_sb    0.015143
16                treelag_2_ns    0.014533
17                treelag_2_os    0.014492
18                treelag_1_ns    0.014380
19                treelag_1_os    0.014347
20             spei1_gs_prev10    0.013061
21        spei1_gs_prev10_anom    0.012437
22      spt

In [3]:
print(top_50_features)

['ged_sb', 'decay_ged_ns_1', 'sptime_dist_k001_ged_ns', 'splag_1_decay_ged_sb_1', 'sptime_dist_k001_ged_os', 'sptime_dist_k10_ged_ns', 'sptime_dist_k10_ged_os', 'sptime_dist_k1_ged_ns', 'wdi_nv_agr_totl_kd', 'spei_48_detrend', 'ged_sb_decay_12_time_since', 'spei1_gsm_detrend', 'ged_gte_1', 'mov_avg_6_ged_best_sb', 'mov_sum_6_ged_best_sb', 'sptime_dist_k1_ged_sb', 'treelag_2_ns', 'treelag_2_os', 'treelag_1_ns', 'treelag_1_os', 'spei1_gs_prev10', 'spei1_gs_prev10_anom', 'sptime_dist_k10_ged_sb', 'sptime_dist_k001_ged_sb', 'ged_sb_tlag_1_splag_1', 'tlag1_spei1_gsm', 'sptime_dist_k1_ged_os', 'mov_avg_12_ged_best_sb', 'mov_sum_12_ged_best_sb', 'ged_sb_tlag_1', 'mov_sum_36_ged_best_sb', 'spei1gsy_lowermedian_count', 'mov_avg_36_ged_best_sb', 'spei1_gsm_cv_anom', 'treelag_2_sb', 'treelag_1_sb', 'count_moder_drought_prev10', 'ged_sb_tlag_7', 'ged_sb_tlag_2', 'ged_sb_tlag_3', 'decay_ged_sb_25', 'tlag_12_crop_sum', 'ged_sb_tlag_4', 'ged_sb_tlag_5', 'ged_sb_tlag_9', 'ged_sb_tlag_8', 'tlag_12_harv

In [4]:
transformer_input

{-3.0866262643659366: array([[-0.023420489678843835, -0.1710966631236406, 4.764483199454078,
         ..., -0.023287209808973492, Timestamp('2018-01-01 00:00:00'),
         -3.0866262643659366],
        [-0.023420489678843835, -0.17109666312548963, 4.765135518014728,
         ..., -0.023287209808973492, Timestamp('2018-02-01 00:00:00'),
         -3.0866262643659366],
        [-0.023420489678843835, -0.17109666312723487, 4.765789709473987,
         ..., -0.023287209808973492, Timestamp('2018-03-01 00:00:00'),
         -3.0866262643659366],
        ...,
        [-0.023420489678843835, -0.17109666315383645, 4.79421636528054,
         ..., -0.023287209808973492, Timestamp('2021-08-01 00:00:00'),
         -3.0866262643659366],
        [-0.023420489678843835, -0.17109666315399072, 4.794948645917301,
         ..., -0.023287209808973492, Timestamp('2021-09-01 00:00:00'),
         -3.0866262643659366],
        [-0.023420489678843835, -0.1710966631541363, 4.795682771736554,
         ..., -0.0232

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
# To inspect first group's datetime handling:
first_gid, first_grp = next(iter(transformer_input.items()))
print(pd.to_datetime(first_grp[:, -2]))  # Check what this outputs to verify correct handling


DatetimeIndex(['2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-01-01', '2020-02-01', '2020-02-01',
               '2020-03-01', '2020-03-01', '2020-04-01', '2020-04-01',
               '2020-05-01', '2020-05-01', '2020-06-01', '2020-06-01',
               '2020-07-01', '2020-07-01', '2020-08-01', '2020-08-01',
               '2020-09-01', '2020-09-01', '2020-10-01', '2020-10-01',
               '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01',
               '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01'],
     

In [11]:
# Final Working Code
import torch
import torch.nn as nn
import numpy as np
import math

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class TransformerDataset(Dataset):
    def __init__(self, data, date_encodings):
        self.data = data
        self.date_encodings = date_encodings

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        gid = list(self.data.keys())[idx]
        features = np.array(self.data[gid][:, :-2], dtype=np.float32)  # Exclude the last two columns if last is target and second last is date
        target = np.array(self.data[gid][:, -1], dtype=np.float32)
        dates = self.date_encodings[gid]
        return torch.tensor(features), torch.tensor(dates), torch.tensor(target)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x, dates):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TemporalTransformerEncoder(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dim_feedforward=256, dropout=0.1):
        super(TemporalTransformerEncoder, self).__init__()
        self.input_embedding = nn.Linear(input_size, d_model // 2)
        self.pos_encoder = PositionalEncoding(d_model // 2, dropout=dropout)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model // 2, nhead=nhead // 2, dim_feedforward=dim_feedforward // 2, dropout=dropout, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_layer = nn.Linear(d_model // 2, 1)

    def forward(self, x, dates):
        x = self.input_embedding(x)
        x = self.pos_encoder(x, dates)
        x = self.transformer_encoder(x)
        output = self.output_layer(x)
        return output.squeeze(-1)

# Proper DataLoader initialization
dataset = TransformerDataset(transformer_input, date_encodings)
train_set, val_set = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_loader = DataLoader([dataset[i] for i in train_set], batch_size=64, shuffle=True)
val_loader = DataLoader([dataset[i] for i in val_set], batch_size=64, shuffle=False)

# Initialize the model
model = TemporalTransformerEncoder(input_size=len(top_50_features), d_model=4, nhead=3, num_layers=8)
criterion = nn.MSELoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(100):
    for x, dates, target in train_loader:
        optimizer.zero_grad()
        output = model(x, dates)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Validation
    val_loss = 0
    for x, dates, target in val_loader:
        output = model(x, dates)
        val_loss += criterion(output, target).item()
    val_loss /= len(val_loader)

    if val_loss <= 1.1:
        print(f"Early Stopping at Epoch :{epoch+1}, Loss : {val_loss}")
        break

    print(f'Epoch: {epoch+1},Loss: {val_loss}')

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch: 1,Loss: 1.1520579907952286
Epoch: 2,Loss: 1.1267655113848245
Epoch: 3,Loss: 1.1124033637163115
Epoch: 4,Loss: 1.102899935187363
Early Stopping at Epoch :5, Loss : 1.0917014334259965


In [9]:
class TemporalTransformerEncoder(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dim_feedforward=512, dropout=0.1):
        super(TemporalTransformerEncoder, self).__init__()
        self.input_embedding = nn.Linear(input_size, d_model)
        self.positional_encoder = nn.Embedding(int(np.max([np.max(dates) for dates in date_encodings.values()])) + 1, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_layer = nn.Linear(d_model, 1)
    
    def forward(self, x, dates):
        x = self.input_embedding(x)
        dates = dates.long()
        pos_encoding = self.positional_encoder(dates)
        x += pos_encoding
        x = self.transformer_encoder(x)
        output = self.output_layer(x)
        return output.squeeze(-1)

# Initialize the model with multi-head architecture
num_heads = 4  # Specify the number of attention heads
model = TemporalTransformerEncoder(input_size=len(top_50_features), d_model=4, nhead=num_heads, num_layers=1)

# Adjust data loading for multiple heads
class TransformerDataset(Dataset):
    def __init__(self, data, date_encodings):
        self.data = data
        self.date_encodings = date_encodings
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        gid = list(self.data.keys())[idx]
        features = np.array(self.data[gid][:, :-2], dtype=np.float32)  # Exclude the last two columns
        target = np.array(self.data[gid][:, -1], dtype=np.float32)
        dates = np.array(self.date_encodings[gid], dtype=np.long)
        return torch.tensor(features), torch.tensor(dates), torch.tensor(target)

# Corrected forward method in TemporalTransformerEncoder
def forward(self, x, dates):
    x = self.input_embedding(x)
    pos_encoding = self.positional_encoder(dates.long())  # Ensure dates is already a tensor
    x += pos_encoding
    x = self.transformer_encoder(x)
    output = self.output_layer(x)
    return output.squeeze(-1)


# Training loop with multi-head support
batch_size = 32  # Define batch size
dataset = TransformerDataset(transformer_input, date_encodings)

train_set, val_set = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_loader = DataLoader([dataset[i] for i in train_set], batch_size=32, shuffle=True)
val_loader = DataLoader([dataset[i] for i in val_set], batch_size=32, shuffle=False)

# Training loop with multi-head support
for epoch in tqdm(range(100)):
    model.train()
    for batch_idx, (features_batch, dates_batch, targets_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(features_batch, dates_batch)
        loss = criterion(outputs, targets_batch)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_losses = []
    with torch.no_grad():
        for features_batch, dates_batch, targets_batch in val_loader:
            outputs = model(features_batch, dates_batch)
            val_loss = criterion(outputs, targets_batch)
            val_losses.append(val_loss.item())
    val_loss_avg = np.mean(val_losses)
    print(f'Epoch {epoch+1}: Val Loss: {val_loss_avg:.4f}')




: 

In [3]:
class TransformerDataset(Dataset):
    def __init__(self, data, date_encodings):
        self.data = data
        self.date_encodings = date_encodings
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        gid = list(self.data.keys())[idx]
        features = np.array(self.data[gid][:, :-2], dtype=np.float32)  # Exclude the last two columns if last is target and second last is date
        target = np.array(self.data[gid][:, -1], dtype=np.float32)
        dates = self.date_encodings[gid]
        return torch.tensor(features), torch.tensor(dates), torch.tensor(target)

In [8]:
# Proper DataLoader initialization
dataset = TransformerDataset(transformer_input, date_encodings)

train_set, val_set = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_loader = DataLoader([dataset[i] for i in train_set], batch_size=32, shuffle=True)
val_loader = DataLoader([dataset[i] for i in val_set], batch_size=32, shuffle=False)

In [9]:
# Define the transformer model
class TemporalTransformerEncoder(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dim_feedforward=512, dropout=0.1):
        super(TemporalTransformerEncoder, self).__init__()
        self.input_embedding = nn.Linear(input_size, d_model)
        self.positional_encoder = nn.Embedding(int(np.max([np.max(dates) for dates in date_encodings.values()])) + 1, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_layer = nn.Linear(d_model, 1)
    
    def forward(self, x, dates):
        x = self.input_embedding(x)
        dates = dates.long()
        pos_encoding = self.positional_encoder(dates)
        x += pos_encoding
        x = self.transformer_encoder(x)
        output = self.output_layer(x)
        return output.squeeze(-1)

In [10]:
# Initialize the model
model = TemporalTransformerEncoder(input_size=len(top_50_features), d_model=4, nhead=1, num_layers=1)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

/opt/homebrew/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [11]:
# Training loop
# Training loop
for epoch in tqdm(range(100)):
    model.train()
    for i, (features, dates, targets) in enumerate(train_loader):
        try:
            optimizer.zero_grad()
            outputs = model(features.squeeze(0), dates.squeeze(0))
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        except Exception as e:
            print(f"Error at iteration {i} of epoch {epoch}: {str(e)}")
            break
    # Validation loop
    model.eval()
    val_losses = []
    with torch.no_grad():
        for features, dates, targets in val_loader:
            outputs = model(features.squeeze(0), dates.squeeze(0))
            val_loss = criterion(outputs, targets)
            val_losses.append(val_loss.item())
    val_loss_avg = np.mean(val_losses)
    print(f'Epoch {epoch+1}: Train Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}')


  0%|          | 0/100 [00:00<?, ?it/s]

: 

In [ ]:
for epoch in tqdm(range(100)):
   model.eval()
    val_losses = []
    with torch.no_grad():
        for features, dates, targets in val_loader:
            outputs = model(features.squeeze(0), dates.squeeze(0))
            val_loss = criterion(outputs, targets)
            val_losses.append(val_loss.item())
    val_loss_avg = np.mean(val_losses)
    print(f'Epoch {epoch+1}: Train Loss: {loss.item():.4f}, Val Loss: {val_loss_avg:.4f}')